Definimos funciones auxiliares para elgamal y operaciones en grupos

In [11]:
import random
from math import fmod


def _is_natural_power(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()

    while pow(2, search_exponent) <= n:

        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1

            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True

        search_exponent += 1

    return False


def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    r, s, t = _extended_euclid_base(b, a)
    return r, t, s


def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t


def _is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if _is_natural_power(n):
        return False

    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False

    return found_negative

def _modinv(a, m):
    g, x, y = _extended_euclid(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [10]:
class EllipticCurve:
    def __init__(self, A, B, p):
        # Raise exceptions
        if not _is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")

        class Element:
            def __init__(self, x, y=None):
                if x < 0 or x > p - 1:
                    raise Exception(f"x={x} is not in the range 0,...,{p - 1}")
                if y != None:
                    if y < 1 or y > p - 1:
                        raise Exception(f"y={y} is not in the range 1,...,{p - 1}")
                    elif ((y ** 2) - (x ** 3 + A * x + B)) % p:
                        print(((y ** 2) - (x ** 3 + A * x + B)))
                        raise Exception(f"y(x,y)={x, y} is not in the elliptic curve")
                self.x = x
                self.y = y

            def __eq__(self, other_element):
                if self.y is None and other_element.y is None:
                    return True
                return self.x == other_element.x and self.y == other_element.y

            def __mul__(self, other_element):
                if self.y == None:
                    return other_element
                if other_element.y == None:
                    return self
                elif self.x != other_element.x:
                    s = (other_element.y - self.y) * (_modinv(((other_element.x - self.x) % p), p)) % p
                    # s = ((other_element.y - self.y) / (other_element.x - self.x)) % p
                    x3 = (s * s - self.x - other_element.x) % p
                    y3 = (s * (self.x - x3) - self.y) % p
                    return Element(x3, y3)
                elif self.x == other_element.x and self.y != other_element.y:
                    return Element(self.x)
                elif self.x == other_element.x and self.y == other_element.y and self.y != 0:
                    s = ((3 * self.x * self.x) + A) * (_modinv((2 * self.y), p)) % p
                    # s = ((3 * (self.x ** 2) + A) / (2 * self.y)) % p
                    x3 = (s * s - self.x - other_element.x) % p
                    y3 = (s * (self.x - x3) - self.y) % p
                    return Element(x3, y3)
                elif self.x == other_element.x and self.y == other_element.y and self.y == 0:
                    return Element(self.x)

            def __pow__(self, exponent):
                binrep = bin(exponent)[2:]
                res = Element(1,None)
                for bit in binrep:
                    res = res*res
                    if bit == "1":
                        res = res*self
                return res

            def __str__(self):
                return f"{self.x, self.y}"

        self.element_class = Element

    def get_identity(self):
        return self.get_element(1, None)

    def get_element(self, x, y):
        return self.element_class(x, y)


class ZpStar:

    def __init__(self, p):
        if not _is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")

        class Element:
            def __init__(self, value):
                if value < 1 or value > p - 1:
                    raise Exception(f"value={value} is not in the range 1,...,{p - 1}")
                self.value = value

            # Allows to compare elements with ==
            def __eq__(self, other_element):
                return self.value == other_element.value

            # Allows to operate elements with *
            def __mul__(self, other_element):
                return Element((self.value * other_element.value) % p)

            # Allows to use ** as exponentiation
            def __pow__(self, exponent):
                return Element(pow(self.value, exponent, p))

            # Allows to use str(e) to transform element e into a string
            def __str__(self):
                return str(self.value)

        self.element_class = Element

    def get_identity(self):
        return self.get_element(1)

    def get_element(self, n):
        return self.element_class(n)


class ElGamalReceiver:
    def __init__(self, group, generator, group_order):
        self.group = group
        self.order = group_order
        self.generator = generator
        # Is the order of the generator correct? For this we check that
        # 1. The group order is prime; and
        # 2. The generator to the power of the order is 1.
        if not _is_probably_prime(group_order):
            raise Exception(f"p={group_order} is not a prime number")

        if not pow(generator, group_order) == self.group.get_identity():
            raise Exception(f"p={generator} bad")
        # The secret key is simply a random scalar in the correct range
        self.x = random.randint(1, group_order - 1)

    def get_public_key(self):
        # The public key must be a tuple containing the group,
        # the generator, the order of the generator, and the
        # generator to the power secret_key
        return (self.group, self.generator, self.order, pow(self.generator, self.x))

    def decrypt(self, ciphertext):
        # Receives a pair representing a ciphertext.
        ms, gy = ciphertext
        s_inverso = pow(gy, self.order - self.x)
        return ms * s_inverso


class ElGamalSender:
    def __init__(self, pubkey):
        self.group, self.generator, self.group_order, self.gx = pubkey
        self.y = random.randint(1, self.group_order - 1)

    def encrypt(self, group_element):
        # We are only encrypting single group elements.
        # According to ElGamal, the ciphertext is a pair.
        s = pow(self.gx, self.y)
        return group_element * s, pow(self.generator, self.y)